In [12]:
# ItemKNN Recommender with Diversity Reranking for LastFM

import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import math
import random
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

#################################
# ITEMKNN RECOMMENDER IMPLEMENTATION
#################################

class ItemKNNRecommender:
    def __init__(self, k_neighbors=50, random_state=42):
        """
        Item-based K-Nearest Neighbors recommender algorithm
        
        Parameters:
        - k_neighbors: number of nearest neighbors to consider
        - random_state: seed for reproducibility
        """
        self.k_neighbors = k_neighbors
        self.random_state = random_state
        np.random.seed(random_state)
        
    def fit(self, user_item_matrix):
        """
        Train the ItemKNN model on the user-item matrix
        
        Parameters:
        - user_item_matrix: scipy sparse matrix with user-item interactions
        
        Returns:
        - self
        """
        self.user_item_matrix = user_item_matrix
        self.n_users, self.n_items = user_item_matrix.shape
        
        # Create a dictionary of items each user has interacted with
        self.user_items = defaultdict(set)
        for user, item in zip(*self.user_item_matrix.nonzero()):
            self.user_items[user].add(item)
        
        # Compute item-item similarity matrix using cosine similarity
        print("Computing item-item similarity matrix...")
        self.item_similarity = cosine_similarity(user_item_matrix.T)
        
        # Exclude self-similarity (set diagonal to 0)
        np.fill_diagonal(self.item_similarity, 0)
        
        # For each item, keep only top k_neighbors
        if self.k_neighbors < self.n_items:
            for i in range(self.n_items):
                sim_scores = self.item_similarity[i]
                top_k_idx = np.argsort(sim_scores)[::-1][:self.k_neighbors]
                mask = np.ones(self.n_items, dtype=bool)
                mask[top_k_idx] = False
                self.item_similarity[i, mask] = 0
                
        print(f"ItemKNN model trained with {self.k_neighbors} neighbors")
        return self
    
    def recommend(self, user_id, n=10, exclude_seen=True):
        """
        Generate item recommendations for a user
        
        Parameters:
        - user_id: user index
        - n: number of recommendations to generate
        - exclude_seen: whether to exclude items the user has already interacted with
        
        Returns:
        - list of n recommended item indices
        """
        if user_id in self.user_items:
            user_items = list(self.user_items[user_id])
        else:
            items = np.arange(self.n_items)
            np.random.shuffle(items)
            return items[:n]
        
        scores = np.zeros(self.n_items)
        for item in user_items:
            scores += self.item_similarity[item]
        if len(user_items) > 0:
            scores /= len(user_items)
        if exclude_seen:
            scores[user_items] = -np.inf
        top_items = np.argsort(scores)[::-1][:n]
        return top_items

#################################
# RERANKER IMPLEMENTATION
#################################

class SimpleReranker:
    """
    Simple reranker that balances original scores with diversity.
    """
    def __init__(self, model, alpha=0.7):
        """
        Initialize reranker
        
        Parameters:
        - model: trained recommender model
        - alpha: weight for original scores (0 to 1); higher alpha means more focus on accuracy
        """
        self.model = model
        self.alpha = alpha
        
        # Calculate item popularity
        self.item_popularity = np.zeros(model.n_items)
        for user in range(model.n_users):
            if user in model.user_items:
                for item in model.user_items[user]:
                    self.item_popularity[item] += 1
        max_pop = np.max(self.item_popularity)
        if max_pop > 0:
            self.norm_popularity = self.item_popularity / max_pop
        else:
            self.norm_popularity = np.zeros_like(self.item_popularity)
    
    def rerank(self, user_id, n=10):
        """
        Generate reranked recommendations.
        """
        if user_id in self.model.user_items:
            user_items = list(self.model.user_items[user_id])
        else:
            items = np.arange(self.model.n_items)
            np.random.shuffle(items)
            return items[:n]
        
        original_scores = np.zeros(self.model.n_items)
        for item in user_items:
            original_scores += self.model.item_similarity[item]
        if len(user_items) > 0:
            original_scores /= len(user_items)
        original_scores[user_items] = -np.inf
        
        candidates = np.argsort(original_scores)[::-1][:n*3]
        selected = []
        while len(selected) < n and candidates.size > 0:
            best_score = -np.inf
            best_item = None
            for item in candidates:
                if item in selected:
                    continue
                score_orig = original_scores[item]
                diversity_score = 0
                if selected:
                    similarities = [self.model.item_similarity[item, sel] for sel in selected]
                    if similarities:
                        avg_sim = np.mean(similarities)
                        diversity_score = 1 - avg_sim
                novelty_score = 1 - self.norm_popularity[item]
                combined_score = (
                    self.alpha * score_orig +
                    (1 - self.alpha) * 0.5 * diversity_score +
                    (1 - self.alpha) * 0.5 * novelty_score
                )
                if combined_score > best_score:
                    best_score = combined_score
                    best_item = item
            if best_item is None:
                break
            selected.append(best_item)
            candidates = candidates[candidates != best_item]
        return selected

class MMRReranker:
    """
    Maximum Marginal Relevance (MMR) Reranker.
    
    Balances relevance and diversity by selecting items that maximize:
    MMR = λ * rel(i) - (1-λ) * max(sim(i,j)) for j in selected items.
    """
    def __init__(self, model, lambda_param=0.7):
        """
        Initialize the MMR reranker.
        
        Parameters:
        - model: trained ItemKNN model
        - lambda_param: trade-off parameter between relevance and diversity (0-1)
        """
        self.model = model
        self.lambda_param = lambda_param
        
    def calculate_item_similarity(self, item1, item2):
        return self.model.item_similarity[item1, item2]
    
    def rerank(self, user_id, n=10, candidate_size=100):
        if user_id in self.model.user_items:
            user_items = list(self.model.user_items[user_id])
        else:
            items = np.arange(self.model.n_items)
            np.random.shuffle(items)
            return items[:n]
        
        relevance_scores = np.zeros(self.model.n_items)
        for item in user_items:
            relevance_scores += self.model.item_similarity[item]
        if len(user_items) > 0:
            relevance_scores /= len(user_items)
        relevance_scores[user_items] = -np.inf
        
        candidates = np.argsort(relevance_scores)[::-1][:candidate_size]
        candidate_scores = relevance_scores[candidates]
        min_score = np.min(candidate_scores)
        max_score = np.max(candidate_scores)
        score_range = max_score - min_score
        if score_range > 0:
            normalized_scores = (candidate_scores - min_score) / score_range
        else:
            normalized_scores = np.zeros_like(candidate_scores)
        
        selected = []
        if candidates.size > 0:
            selected.append(candidates[np.argmax(normalized_scores)])
            remaining_candidates = set(candidates) - set(selected)
        else:
            remaining_candidates = set()
        
        while len(selected) < n and remaining_candidates:
            max_mmr = -np.inf
            max_item = None
            for item in remaining_candidates:
                item_idx = np.where(candidates == item)[0][0]
                relevance = normalized_scores[item_idx]
                max_sim = 0
                for sel in selected:
                    sim = self.calculate_item_similarity(item, sel)
                    max_sim = max(max_sim, sim)
                mmr_score = self.lambda_param * relevance - (1 - self.lambda_param) * max_sim
                if mmr_score > max_mmr:
                    max_mmr = mmr_score
                    max_item = item
            if max_item is not None:
                selected.append(max_item)
                remaining_candidates.remove(max_item)
            else:
                break
        return selected

#################################
# EVALUATION METRICS
#################################

def calculate_ndcg(recommended_items, relevant_items, relevant_scores, k=None):
    if k is None:
        k = len(recommended_items)
    else:
        k = min(k, len(recommended_items))
    
    relevance_map = {}
    for item_id, score in zip(relevant_items, relevant_scores):
        try:
            score_float = float(score)
            score_float = min(score_float, 10.0)
            relevance_map[item_id] = score_float
        except (ValueError, TypeError):
            relevance_map[item_id] = 0.0
    
    dcg = 0
    for i, item_id in enumerate(recommended_items[:k]):
        if item_id in relevance_map:
            rel = float(relevance_map[item_id])
            dcg += (2 ** rel - 1) / np.log2(i + 2)
    
    sorted_relevant = []
    for item_id, score in zip(relevant_items, relevant_scores):
        try:
            score_float = float(score)
            score_float = min(score_float, 10.0)
            sorted_relevant.append((item_id, score_float))
        except (ValueError, TypeError):
            sorted_relevant.append((item_id, 0.0))
    sorted_relevant.sort(key=lambda x: x[1], reverse=True)
    
    idcg = 0
    for i, (item_id, rel) in enumerate(sorted_relevant[:k]):
        idcg += (2 ** float(rel) - 1) / np.log2(i + 2)
    
    return dcg / idcg if idcg > 0 else 0

def calculate_precision(recommended_items, relevant_items):
    num_relevant = sum(1 for item in recommended_items if item in relevant_items)
    return num_relevant / len(recommended_items) if recommended_items else 0

def calculate_recall(recommended_items, relevant_items):
    num_relevant = sum(1 for item in recommended_items if item in relevant_items)
    return num_relevant / len(relevant_items) if relevant_items else 0

def calculate_diversity_metrics(recommendations, item_popularity, total_items, tail_items=None):
    rec_counts = Counter(recommendations)
    item_coverage = len(rec_counts) / total_items
    sorted_counts = sorted(rec_counts.values())
    n = len(sorted_counts)
    if n == 0:
        gini_index = 0
    else:
        cumulative_sum = sum((i + 1) * count for i, count in enumerate(sorted_counts))
        gini_index = (2 * cumulative_sum) / (n * sum(sorted_counts)) - (n + 1) / n
    rec_total = sum(rec_counts.values())
    probabilities = [count / rec_total for count in rec_counts.values()]
    entropy = -sum(p * np.log2(p) for p in probabilities if p > 0)
    max_entropy = np.log2(min(total_items, rec_total))
    normalized_entropy = entropy / max_entropy if max_entropy > 0 else 0
    
    if tail_items is None:
        sorted_pop = np.argsort(item_popularity)
        num_tail = int(len(sorted_pop) * 0.2)
        tail_items = set(sorted_pop[:num_tail])
    tail_rec = sum(1 for item in recommendations if item in tail_items)
    tail_percentage = tail_rec / len(recommendations) if recommendations else 0
    
    metrics = {
        'item_coverage': item_coverage,
        'gini_index': gini_index,
        'shannon_entropy': normalized_entropy,
        'tail_percentage': tail_percentage
    }
    return metrics, tail_items

#################################
# HELPER FUNCTIONS
#################################

def load_lastfm(path="lastfm/lastfm.inter"):
    print("Loading LastFM dataset...")
    encodings_to_try = ['utf-8', 'latin-1', 'ISO-8859-1', 'cp1252']
    
    def generate_sample_data():
        print("Generating sample LastFM data for demonstration purposes...")
        np.random.seed(42)
        n_users = 100
        n_items = 50
        n_ratings = 1000
        user_ids = [f"user_{i}" for i in range(n_users)]
        item_ids = [f"artist_{i}" for i in range(n_items)]
        random_users = np.random.choice(user_ids, size=n_ratings)
        random_items = np.random.choice(item_ids, size=n_ratings)
        random_ratings = np.random.uniform(1, 5, size=n_ratings)
        constant_timestamp = np.full(n_ratings, 1111111111)
        sample_df = pd.DataFrame({
            'user_id': random_users,
            'item_id': random_items,
            'rating': random_ratings,
            'timestamp': constant_timestamp
        })
        dummy_df = pd.DataFrame(columns=['item_id', 'name', 'tags'])
        print(f"Generated sample data with {len(sample_df)} ratings from {sample_df['user_id'].nunique()} users on {sample_df['item_id'].nunique()} artists")
        return sample_df, dummy_df
    
    for encoding in encodings_to_try:
        try:
            print(f"Trying to load LastFM data with {encoding} encoding...")
            columns = ['user_id', 'artist_id', 'weight', 'tag_value']
            raw_df = pd.read_csv(path, sep='\t', names=columns, encoding=encoding)
            raw_df = raw_df.rename(columns={'artist_id': 'item_id', 'weight': 'rating'})
            raw_df['timestamp'] = 1111111111
            raw_df = raw_df.drop(columns=['tag_value'])
            dummy_df = pd.DataFrame(columns=['item_id', 'name', 'tags'])
            if len(raw_df) > 0:
                print(f"Successfully loaded LastFM dataset with {encoding} encoding")
                print(f"Loaded {len(raw_df)} interactions from {raw_df['user_id'].nunique()} users on {raw_df['item_id'].nunique()} artists")
                return raw_df, dummy_df
        except Exception as e:
            print(f"Error loading LastFM dataset with {encoding} encoding: {str(e)}")
            continue
    print("All attempts to load the LastFM dataset failed. Generating sample data instead.")
    return generate_sample_data()

def create_user_item_matrix(ratings_df):
    user_ids = ratings_df['user_id'].unique()
    item_ids = ratings_df['item_id'].unique()
    user_mapping = {user_id: i for i, user_id in enumerate(user_ids)}
    item_mapping = {item_id: i for i, item_id in enumerate(item_ids)}
    rows = ratings_df['user_id'].map(user_mapping)
    cols = ratings_df['item_id'].map(item_mapping)
    data = np.ones(len(ratings_df))
    user_item_matrix = csr_matrix((data, (rows, cols)),
                                  shape=(len(user_mapping), len(item_mapping)))
    return user_item_matrix, user_mapping, item_mapping

#################################
# COMPREHENSIVE EVALUATION
#################################

def comprehensive_evaluation_multiple_rerankers(k=10, sample_size=None):
    """
    Run a comprehensive evaluation measuring both accuracy and diversity
    for multiple rerankers.
    """
    print("=" * 80)
    print(f"COMPREHENSIVE EVALUATION WITH MULTIPLE RERANKERS (k={k})")
    print("=" * 80)
    
    # Load and prepare data
    print("\nLoading LastFM dataset...")
    ratings_df, dummy_df = load_lastfm()
    
    print("Splitting data for evaluation...")
    value_counts = ratings_df['user_id'].value_counts()
    if value_counts.min() >= 2:
        print("Using stratified sampling...")
        train_df, test_df = train_test_split(
            ratings_df, test_size=0.2, stratify=ratings_df['user_id'], random_state=42
        )
    else:
        print("Using random sampling (some users have only 1 rating)...")
        train_df, test_df = train_test_split(
            ratings_df, test_size=0.2, random_state=42
        )
    
    print("Creating user-item matrix...")
    user_item_matrix, user_mapping, item_mapping = create_user_item_matrix(train_df)
    reverse_user_mapping = {v: k for k, v in user_mapping.items()}
    reverse_item_mapping = {v: k for k, v in item_mapping.items()}
    
    # Build test ground truth
    test_relevant_items = defaultdict(list)
    test_relevant_scores = defaultdict(list)
    for _, row in test_df.iterrows():
        uid, iid, rating = row['user_id'], row['item_id'], row['rating']
        if uid in user_mapping and iid in item_mapping:
            test_relevant_items[uid].append(iid)
            test_relevant_scores[uid].append(rating)
    
    # Train model
    print("\nTraining ItemKNN model...")
    model = ItemKNNRecommender(k_neighbors=50)
    model.fit(user_item_matrix)
    
    # Initialize rerankers
    print("\nInitializing rerankers...")
    simple_reranker = SimpleReranker(model=model, alpha=0.7)
    mmr_reranker = MMRReranker(model=model, lambda_param=0.7)
    
    rerankers = {
        "Original": None,
        "Simple Reranker": simple_reranker,
        "MMR Reranker": mmr_reranker
    }
    
    # Select evaluation users
    if sample_size is not None and sample_size < len(test_relevant_items):
        eval_users = random.sample(list(test_relevant_items.keys()), sample_size)
    else:
        eval_users = list(test_relevant_items.keys())
    print(f"\nEvaluating {len(eval_users)} users...")
    
    # Collect metrics for each reranker
    results = {name: {'ndcg': [], 'precision': [], 'recall': []} for name in rerankers}
    all_recs = {name: [] for name in rerankers}
    
    for uid in eval_users:
        if not test_relevant_items[uid]:
            continue
        user_idx = user_mapping[uid]
        for name, reranker in rerankers.items():
            if reranker is None:
                rec_idx = model.recommend(user_idx, n=k)
            else:
                rec_idx = reranker.rerank(user_idx, n=k)
            rec = [reverse_item_mapping[idx] for idx in rec_idx]
            all_recs[name].extend(rec_idx)
            results[name]['ndcg'].append(calculate_ndcg(rec, test_relevant_items[uid], test_relevant_scores[uid]))
            results[name]['precision'].append(calculate_precision(rec, test_relevant_items[uid]))
            results[name]['recall'].append(calculate_recall(rec, test_relevant_items[uid]))
    
    # Calculate average accuracy metrics
    acc_metrics = {}
    for name in rerankers:
        acc_metrics[name] = {
            f'ndcg@{k}': np.mean(results[name]['ndcg']),
            f'precision@{k}': np.mean(results[name]['precision']),
            f'recall@{k}': np.mean(results[name]['recall'])
        }
    
    # Calculate diversity metrics using all recommendations from each approach
    # First, determine item popularity from the training data:
    item_popularity = np.zeros(model.n_items)
    for user in range(model.n_users):
        if user in model.user_items:
            for item in model.user_items[user]:
                item_popularity[item] += 1
    div_metrics = {}
    orig_div, tail_items = calculate_diversity_metrics(
        recommendations=all_recs["Original"],
        item_popularity=item_popularity,
        total_items=model.n_items
    )
    div_metrics["Original"] = orig_div
    for name in ["Simple Reranker", "MMR Reranker"]:
        div_metrics[name], _ = calculate_diversity_metrics(
            recommendations=all_recs[name],
            item_popularity=item_popularity,
            total_items=model.n_items,
            tail_items=tail_items
        )
    
    # Print Accuracy Metrics Comparison
    print("\n" + "="*30 + " ACCURACY METRICS " + "="*30)
    header = f"{'Metric':<15} {'Original':<20} {'Simple Reranker':<20} {'MMR Reranker':<20}"
    print(header)
    print("-" * len(header))
    for metric in [f'ndcg@{k}', f'precision@{k}', f'recall@{k}']:
        orig_val = acc_metrics["Original"][metric]
        simple_val = acc_metrics["Simple Reranker"][metric]
        mmr_val = acc_metrics["MMR Reranker"][metric]
        simple_change = ((simple_val - orig_val)/orig_val * 100) if orig_val > 0 else float('inf')
        mmr_change = ((mmr_val - orig_val)/orig_val * 100) if orig_val > 0 else float('inf')
        print(f"{metric:<15} {orig_val:.4f}{'':<10} {simple_val:.4f} ({simple_change:+.1f}%){'':<5} {mmr_val:.4f} ({mmr_change:+.1f}%)")
    
    # Print Diversity Metrics Comparison
    print("\n" + "="*30 + " DIVERSITY METRICS " + "="*30)
    header = f"{'Metric':<20} {'Original':<20} {'Simple Reranker':<20} {'MMR Reranker':<20}"
    print(header)
    print("-" * len(header))
    for metric in ['item_coverage', 'gini_index', 'shannon_entropy', 'tail_percentage']:
        orig_val = div_metrics["Original"][metric]
        simple_val = div_metrics["Simple Reranker"][metric]
        mmr_val = div_metrics["MMR Reranker"][metric]
        simple_change = ((simple_val - orig_val)/orig_val * 100) if orig_val > 0 else float('inf')
        mmr_change = ((mmr_val - orig_val)/orig_val * 100) if orig_val > 0 else float('inf')
        print(f"{metric:<20} {orig_val:.4f}{'':<10} {simple_val:.4f} ({simple_change:+.1f}%){'':<5} {mmr_val:.4f} ({mmr_change:+.1f}%)")
    
    # Print interpretations
    print("\n" + "="*30 + " METRIC INTERPRETATIONS " + "="*30)
    print("Accuracy Metrics:")
    print("- NDCG: Higher is better; measures ranking quality")
    print("- Precision: Higher is better; measures the ratio of relevant items recommended")
    print("- Recall: Higher is better; measures coverage of relevant items")
    print("\nDiversity Metrics:")
    print("- Item Coverage: Higher means more items from the catalog are recommended")
    print("- Gini Index: Lower values indicate more equal recommendation distribution")
    print("- Shannon Entropy: Higher values mean more diverse recommendations")
    print("- Tail Percentage: Higher means more niche items are being recommended")
    
    # Return all results for further use if needed
    return {
        'accuracy': acc_metrics,
        'diversity': div_metrics
    }

# Execute evaluation when run directly
if __name__ == "__main__":
    comprehensive_evaluation_multiple_rerankers(k=10)


COMPREHENSIVE EVALUATION WITH MULTIPLE RERANKERS (k=10)

Loading LastFM dataset...
Loading LastFM dataset...
Trying to load LastFM data with utf-8 encoding...
Successfully loaded LastFM dataset with utf-8 encoding
Loaded 92835 interactions from 1893 users on 17633 artists
Splitting data for evaluation...
Using random sampling (some users have only 1 rating)...
Creating user-item matrix...

Training ItemKNN model...
Computing item-item similarity matrix...
ItemKNN model trained with 50 neighbors

Initializing rerankers...

Evaluating 1870 users...

============================== ACCURACY METRICS ==============================
Metric          Original             Simple Reranker      MMR Reranker        
------------------------------------------------------------------------------
ndcg@10         0.2025           0.1161 (-42.7%)      0.2100 (+3.7%)
precision@10    0.1492           0.0950 (-36.3%)      0.1564 (+4.8%)
recall@10       0.1645           0.1058 (-35.7%)      0.1737 (+5.6%)

=